In [ ]:
import pandas as pd
import numpy as np
import tweepy
import datetime
import pytz
import time

key is secret

In [ ]:
# 取得したキーを格納
# BT = "himitu"
# CK = "himitu"
# CS = "himitu"
# AT = "himitu"



In [ ]:
api = tweepy.Client(bearer_token=BT, consumer_key=CK, consumer_secret=CS, access_token=AT, access_token_secret=AS, wait_on_rate_limit=True)

In [ ]:
def utc2jst(timestamp_utc):
    datetime_utc = datetime.datetime.strptime(timestamp_utc + ".000+0000", "%Y-%m-%dT%H:%M:%SZ.%f%z")
    datetime_jst = datetime_utc.astimezone(datetime.timezone(datetime.timedelta(hours=+9)))
    timestamp_jst = datetime.datetime.strftime(datetime_jst, '%Y-%m-%dT%H:%M:%SZ')
    return timestamp_jst

def jst2utc(timestamp_jst):
    datatime_jst = datetime.datetime.strptime(timestamp_jst + ".000+0000", "%Y-%m-%dT%H:%M:%SZ.%f%z")
    datetime_utc = datatime_jst.astimezone(datetime.timezone(datetime.timedelta(hours=-9)))
    timestamp_utc = datetime.datetime.strftime(datetime_utc, '%Y-%m-%dT%H:%M:%SZ')
    return timestamp_utc

In [ ]:
search_word = "place:japan  has:geo lang:ja"
start_time = "2019-10-05T00:00:00Z"
end_time = "2019-10-06T23:59:59Z"
item_number = 10
#tweets = api.search_all_tweets(query=search_word, end_time=jst2utc(end_time), start_time=jst2utc(start_time), max_results=item_number)

In [ ]:
def make_timestamp(yyyy,mm,dd):
    timestamp =(str(yyyy) + '-' +str(mm)+ '-' +str(dd) +'T00:00:00Z')
    return timestamp

In [ ]:
make_timestamp('2000','02','19')

In [ ]:
tmp = tweepy.Paginator(api.search_all_tweets,
                                query = search_word,
                                user_fields = ['username', 'public_metrics', 'description', 'location',],
                                tweet_fields = ['created_at', 'geo', 'public_metrics', 'text', 'source', 'attachments'],
                                media_fields = ['url', 'type', ],
                                expansions = ['author_id', 'attachments.media_keys'],
                                 end_time=jst2utc(end_time),
                                 start_time=jst2utc(start_time),
                                 max_results=10)

In [ ]:
list_tweets = []
counter = 0
for response in tweepy.Paginator(api.search_all_tweets,
                                query = search_word,
                                user_fields = ['username', 'public_metrics', 'description', 'location',],
                                tweet_fields = ['created_at', 'geo', 'public_metrics', 'text', 'source', 'attachments'],
                                media_fields = ['url', 'type', ],
                                expansions = ['author_id', 'attachments.media_keys'],
                                 end_time=jst2utc(end_time),
                                 start_time=jst2utc(start_time),
                                 max_results=10):
    time.sleep(1)
    list_tweets.append(response)
    break

In [ ]:
result = []
user_dict = {}
# Loop through each response object
for response in list_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username,
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        if tweet.geo != None:
            s = str(tweet['created_at'])
            created_at = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S%z")
            created_at = (str(created_at.year) + '-' + str(created_at.month) + '-' + str(created_at.day) + 'T'+'00:00:00Z')
            created_at = utc2jst(created_at)
            result.append({'author_id': tweet.author_id,
                        'username': author_info['username'],
                        'author_followers': author_info['followers'],
                        'author_tweets': author_info['tweets'],
                        'author_description': author_info['description'],
                        'author_location': author_info['location'],
                        'text': tweet.text,
                        'created_at': created_at,
                        'retweets': tweet.public_metrics['retweet_count'],
                        'replies': tweet.public_metrics['reply_count'],
                        'likes': tweet.public_metrics['like_count'],
                        'quote_count': tweet.public_metrics['quote_count'],
                        'place_id':tweet.geo['place_id']
                        })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [ ]:
df